Week 2.
Attempt at reading in L5 Consesnus Lincs data and applying Metrics to it.

Most content is copied from the cyto-eval demo notebook. Thanks Greg!

In [9]:
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
import plotnine as gg
from cytominer_eval import evaluate
from pycytominer.cyto_utils import infer_cp_features

## Load data

In [2]:
data_dir = '2016_04_01_a549_48hr_batch1_consensus_modz_feature_select_dmso.csv.gz'

In [4]:
df = pd.read_csv(data_dir)

In [16]:
print(df.shape)
#metadata_columns = [x for x in df.columns if (x.startswith("Metadata_"))]
#print(len(metadata_columns))
df.head()

(10752, 528)


,Metadata_Plate_Map_Name,Metadata_cell_id,Metadata_broad_sample,Metadata_pert_well,Metadata_mmoles_per_liter,Metadata_dose_recode,Metadata_time_point,Metadata_moa,Metadata_target,Cells_AreaShape_Area,...,Nuclei_Texture_InverseDifferenceMoment_RNA_10_0,Nuclei_Texture_InverseDifferenceMoment_RNA_20_0,Nuclei_Texture_SumAverage_DNA_20_0,Nuclei_Texture_SumAverage_Mito_5_0,Nuclei_Texture_SumAverage_RNA_20_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_DNA_20_0,Nuclei_Texture_SumEntropy_DNA_5_0,Nuclei_Texture_SumEntropy_RNA_5_0,Nuclei_Texture_Variance_RNA_10_0
0,C-7161-01-LM6-001,A549,BRD-A25234499-001-18-3,B13,10.00000,6,48H,glucocorticoid receptor antagonist,CYP11A1|CYP19A1,0.344670,...,-0.302193,-0.262680,-0.643318,-0.307924,-0.263607,-0.771062,-0.622449,-0.968180,0.087257,0.691044
1,C-7161-01-LM6-001,A549,BRD-A25234499-001-18-3,B14,3.33330,5,48H,glucocorticoid receptor antagonist,CYP11A1|CYP19A1,0.488995,...,-0.498673,0.301335,-0.161695,-2.046360,-0.147213,-0.230816,-0.157423,-0.247806,0.243377,-1.196500
2,C-7161-01-LM6-001,A549,BRD-A25234499-001-18-3,B15,1.11110,4,48H,glucocorticoid receptor antagonist,CYP11A1|CYP19A1,0.505526,...,-0.250714,-0.020691,0.722674,-0.759924,-0.012077,-0.270183,-0.134932,-0.359148,0.276299,-0.796096
3,C-7161-01-LM6-001,A549,BRD-A25234499-001-18-3,B16,0.37037,3,48H,glucocorticoid receptor antagonist,CYP11A1|CYP19A1,0.092069,...,-0.338021,0.568476,-0.552219,-0.770124,-0.391011,-0.667111,-0.517957,-0.674251,0.022910,-0.944185
4,C-7161-01-LM6-001,A549,BRD-A25234499-001-18-3,B17,0.12346,2,48H,glucocorticoid receptor antagonist,CYP11A1|CYP19A1,0.705527,...,-0.277073,0.775823,-0.895731,-0.762473,-0.429907,-0.214208,-0.198268,-0.241933,0.026243,-0.849128


In [35]:
features = infer_cp_features(df)
meta_features = infer_cp_features(df, metadata=True)
len(df.Metadata_broad_sample.unique())

1571

In [20]:
moa_labels = df.Metadata_moa.unique()
len(moa_labels)

602

### Part 2: Replicate reproducibility

`operation="precision_recall"`

To calculate precision recall for every perturbation, we first rank order all perturbations by pairwise correlations.
We use a user-defined `k` as a threshold for calling hits.

The operation is done for all moa ground truths

In [41]:
precision_recall_at_k = []

for moa in moa_labels:
    moa_df = df.query("Metadata_moa == @moa")

    for k in [2, 5, 10, 15, 25]:

        precision_recall_results = evaluate(
            profiles=moa_df,
            features=features,
            meta_features=meta_features,
            replicate_groups=["Metadata_moa", "Metadata_broad_sample"],
            operation="precision_recall",
            similarity_metric="pearson",
            precision_recall_k=k
        ).assign(moa=moa)

        precision_recall_at_k.append(precision_recall_results)

In [42]:
df_precision_recall_at_k = pd.concat(precision_recall_at_k).reset_index(drop=True)
print(df_precision_recall_at_k.shape)
df_precision_recall_at_k.head()

(7855, 6)


,Metadata_moa,Metadata_broad_sample,k,precision,recall,moa
0,glucocorticoid receptor antagonist,BRD-A25234499-001-18-3,2.0,1.0,0.066667,glucocorticoid receptor antagonist
1,glucocorticoid receptor antagonist,BRD-A25234499-001-18-3,5.0,1.0,0.166667,glucocorticoid receptor antagonist
2,glucocorticoid receptor antagonist,BRD-A25234499-001-18-3,10.0,1.0,0.333333,glucocorticoid receptor antagonist
3,glucocorticoid receptor antagonist,BRD-A25234499-001-18-3,15.0,1.0,0.500000,glucocorticoid receptor antagonist
4,glucocorticoid receptor antagonist,BRD-A25234499-001-18-3,25.0,1.0,0.833333,glucocorticoid receptor antagonist


In [49]:
df_res = df_precision_recall_at_k.copy()

In [51]:
df_res.loc[df_res['Metadata_moa'] == "glucocorticoid receptor antagonist"]

,Metadata_moa,Metadata_broad_sample,k,precision,recall,moa
0,glucocorticoid receptor antagonist,BRD-A25234499-001-18-3,2.0,1.0,0.066667,glucocorticoid receptor antagonist
1,glucocorticoid receptor antagonist,BRD-A25234499-001-18-3,5.0,1.0,0.166667,glucocorticoid receptor antagonist
2,glucocorticoid receptor antagonist,BRD-A25234499-001-18-3,10.0,1.0,0.333333,glucocorticoid receptor antagonist
3,glucocorticoid receptor antagonist,BRD-A25234499-001-18-3,15.0,1.0,0.500000,glucocorticoid receptor antagonist
4,glucocorticoid receptor antagonist,BRD-A25234499-001-18-3,25.0,1.0,0.833333,glucocorticoid receptor antagonist


In [59]:
pr_av = []
for k in df_res.k.unique():
    pr_av.append(df_res.loc[df_res.k == k].mean().tolist())

In [61]:
pr_av

[[2.0, 0.5432845321451305, 0.03611289905871458],
 [5.0, 0.5437301082113304, 0.09032469194525412],
 [10.0, 0.5394653087205602, 0.17922778830367803],
 [15.0, 0.5311266709102482, 0.2646723656720743],
 [25.0, 0.5113430935709739, 0.42461307709333834]]

Using only one "replicate_groups = broad sample"

In [63]:
precision_recall_at_k = []

for moa in moa_labels:
    moa_df = df.query("Metadata_moa == @moa")

    for k in [2, 10, 25]:

        precision_recall_results = evaluate(
            profiles=moa_df,
            features=features,
            meta_features=meta_features,
            replicate_groups=["Metadata_broad_sample"],
            operation="precision_recall",
            similarity_metric="pearson",
            precision_recall_k=k
        ).assign(moa=moa)

        precision_recall_at_k.append(precision_recall_results)

In [64]:
df_res_1 = pd.concat(precision_recall_at_k).reset_index(drop=True)
print(df_res_1.shape)
df_res_1.head()

(4713, 5)


,Metadata_broad_sample,k,precision,recall,moa
0,BRD-A25234499-001-18-3,2.0,1.0,0.066667,glucocorticoid receptor antagonist
1,BRD-A25234499-001-18-3,10.0,1.0,0.333333,glucocorticoid receptor antagonist
2,BRD-A25234499-001-18-3,25.0,1.0,0.833333,glucocorticoid receptor antagonist
3,BRD-A26032986-050-02-1,2.0,1.0,0.066667,calmodulin antagonist
4,BRD-K44309363-003-02-1,2.0,1.0,0.066667,calmodulin antagonist


In [69]:
pr_av = []
for k in df_res_1.k.unique():
    pr_av.append(df_res_1.query("k==@k").mean().tolist())
pr_av

[[2.0, 0.5432845321451305, 0.03611289905871458],
 [10.0, 0.5394653087205602, 0.17922778830367803],
 [25.0, 0.5113430935709739, 0.42461307709333834]]

Using only one "replicate_groups = MOA"

In [104]:
precision_recall_at_k = []


for k in [2, 10]:

    precision_recall_results = evaluate(
        profiles=selected_moa,
        features=features,
        meta_features=meta_features,
        replicate_groups=["Metadata_moa"],
        operation="precision_recall",
        similarity_metric="pearson",
        precision_recall_k=k
    )

    precision_recall_at_k.append(precision_recall_results)

KeyboardInterrupt: 

In [100]:
df_res_1 = pd.concat(precision_recall_at_k).reset_index(drop=True)
print(df_res_1.shape)
df_res_1.head()

(4, 4)


,Metadata_moa,k,precision,recall
0,calmodulin antagonist,2.0,0.0,0.000000
1,glucocorticoid receptor antagonist,2.0,0.0,0.000000
2,calmodulin antagonist,10.0,0.4,0.030303
3,glucocorticoid receptor antagonist,10.0,0.2,0.066667


In [101]:
pr_av = []
for k in df_res_1.k.unique():
    pr_av.append(df_res_1.query("k==@k").mean().tolist())
pr_av

[[2.0, 0.0, 0.0], [10.0, 0.30000000000000004, 0.048484848484848485]]

In [102]:
ls = df.Metadata_moa.unique()[:2].tolist()
ls
selected_moa= df.query("Metadata_moa==@ls")


In [103]:
selected_moa


,Metadata_Plate_Map_Name,Metadata_cell_id,Metadata_broad_sample,Metadata_pert_well,Metadata_mmoles_per_liter,Metadata_dose_recode,Metadata_time_point,Metadata_moa,Metadata_target,Cells_AreaShape_Area,...,Nuclei_Texture_InverseDifferenceMoment_RNA_10_0,Nuclei_Texture_InverseDifferenceMoment_RNA_20_0,Nuclei_Texture_SumAverage_DNA_20_0,Nuclei_Texture_SumAverage_Mito_5_0,Nuclei_Texture_SumAverage_RNA_20_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_DNA_20_0,Nuclei_Texture_SumEntropy_DNA_5_0,Nuclei_Texture_SumEntropy_RNA_5_0,Nuclei_Texture_Variance_RNA_10_0
0,C-7161-01-LM6-001,A549,BRD-A25234499-001-18-3,B13,10.000000,6,48H,glucocorticoid receptor antagonist,CYP11A1|CYP19A1,0.344670,...,-0.302193,-0.262680,-0.643318,-0.307924,-0.263607,-0.771062,-0.622449,-0.968180,0.087257,0.691044
1,C-7161-01-LM6-001,A549,BRD-A25234499-001-18-3,B14,3.333300,5,48H,glucocorticoid receptor antagonist,CYP11A1|CYP19A1,0.488995,...,-0.498673,0.301335,-0.161695,-2.046360,-0.147213,-0.230816,-0.157423,-0.247806,0.243377,-1.196500
2,C-7161-01-LM6-001,A549,BRD-A25234499-001-18-3,B15,1.111100,4,48H,glucocorticoid receptor antagonist,CYP11A1|CYP19A1,0.505526,...,-0.250714,-0.020691,0.722674,-0.759924,-0.012077,-0.270183,-0.134932,-0.359148,0.276299,-0.796096
3,C-7161-01-LM6-001,A549,BRD-A25234499-001-18-3,B16,0.370370,3,48H,glucocorticoid receptor antagonist,CYP11A1|CYP19A1,0.092069,...,-0.338021,0.568476,-0.552219,-0.770124,-0.391011,-0.667111,-0.517957,-0.674251,0.022910,-0.944185
4,C-7161-01-LM6-001,A549,BRD-A25234499-001-18-3,B17,0.123460,2,48H,glucocorticoid receptor antagonist,CYP11A1|CYP19A1,0.705527,...,-0.277073,0.775823,-0.895731,-0.762473,-0.429907,-0.214208,-0.198268,-0.241933,0.026243,-0.849128
5,C-7161-01-LM6-001,A549,BRD-A25234499-001-18-3,B18,0.041152,1,48H,glucocorticoid receptor antagonist,CYP11A1|CYP19A1,0.660675,...,-0.196240,0.463537,-0.466923,0.131255,-0.029748,-0.089949,-0.050630,-0.167733,0.423890,-0.865904
6,C-7161-01-LM6-001,A549,BRD-A26032986-050-02-1,C13,10.000000,6,48H,calmodulin antagonist,CALM1,0.006508,...,-0.361908,0.490263,0.749907,-1.321410,-0.160391,-0.928242,-0.681633,-1.068360,-0.446784,-0.177102
7,C-7161-01-LM6-001,A549,BRD-A26032986-050-02-1,C14,3.333300,5,48H,calmodulin antagonist,CALM1,-0.335829,...,-0.122731,0.816301,0.463976,-0.084156,-0.548634,-0.430176,-0.462279,-0.427966,-0.412447,-0.597361
8,C-7161-01-LM6-001,A549,BRD-A26032986-050-02-1,C15,1.111100,4,48H,calmodulin antagonist,CALM1,-0.495838,...,-0.393809,0.783456,0.712816,-0.094821,-0.713905,-0.732230,-0.667627,-0.875021,-0.480377,-0.632410
9,C-7161-01-LM6-001,A549,BRD-A26032986-050-02-1,C16,0.370370,3,48H,calmodulin antagonist,CALM1,0.159910,...,0.132909,1.508950,-0.247320,-0.346166,-0.834109,-0.314153,-0.384395,-0.315538,-1.014570,-1.039960
